## Importing packages

In [65]:
import pickle
import gzip
import numpy
from __future__ import print_function
import tensorflow as tf
import theano.tensor as T
import theano.tensor
#import _pickle as cPickle
import os
os.chdir('D:\\Aegis\\Deep Learning')

## Converting gz file into pickle file

In [66]:
''' with gzip.open('17flower.gz', 'rb') as f:
        flower = list(pickle.load(f))
        write_file=open('flower.pkl','wb')
        write_file.close() '''

" with gzip.open('17flower.gz', 'rb') as f:\n        flower = list(pickle.load(f))\n        write_file=open('flower.pkl','wb')\n        write_file.close() "

In [67]:
pickle_file = 'flower.pkl'
with open(pickle_file, 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')

In [68]:
def shared_dataset(data_xy):

    data_x, data_y = data_xy

    shared_x = theano.shared(numpy.asarray(data_x, dtype=theano.config.floatX))

    shared_y = theano.shared(numpy.asarray(data_y, dtype=theano.config.floatX))

    return shared_x, T.cast(shared_y, 'int32')

In [69]:
test_set_x, test_set_y0 = shared_dataset(test_set)

valid_set_x, valid_set_y0 = shared_dataset(valid_set)

train_set_x, train_set_y0 = shared_dataset(train_set)

print(train_set_x.eval().shape)

print(train_set_y0.eval().shape)

print(valid_set_x.eval().shape)


(680, 784)
(680,)
(339, 784)


## Parameters

In [70]:
learning_rate = 0.001

training_iters = 250000

batch_size = 128

display_step = 10

In [71]:
n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size

## Network Parameters

In [72]:
n_input = 784 # MNIST data input (img shape: 28*28)

n_classes = 17 # MNIST total classes (0-9 digits)

dropout = 0.75 # Dropout, probability to keep units

## change_train_label

In [73]:
train_set_y1 = [[0 for col in range(n_classes)] for row in range(train_set_x.eval().shape[0])]
print(len(train_set_y1))

680


In [74]:
for i in range(train_set_y0.eval().shape[0]):

    temp = train_set_y0.eval()[i]

    train_set_y1[i][temp] = 1

shared_y1 = theano.shared(numpy.asarray(train_set_y1, dtype=theano.config.floatX))

train_set_y = T.cast(shared_y1, 'float64')

## change_valid_label

In [75]:
valid_set_y1 = [[0 for col in range(n_classes)] for row in range(valid_set_x.eval().shape[0])]
print(len(valid_set_y1))

339


In [76]:
for i in range(valid_set_y0.eval().shape[0]):

    temp = valid_set_y0.eval()[i]

    valid_set_y1[i][temp] = 1

shared_y1 = theano.shared(numpy.asarray(valid_set_y1, dtype=theano.config.floatX))

valid_set_y = T.cast(shared_y1, 'float64')

## change_test_label

In [77]:
test_set_y1 = [[0 for col in range(n_classes)] for row in range(test_set_x.eval().shape[0])]
print(len(test_set_y1))

339


In [78]:
for i in range(test_set_y0.eval().shape[0]):

    temp = test_set_y0.eval()[i]

    test_set_y1[i][temp] = 1

shared_y1 = theano.shared(numpy.asarray(test_set_y1, dtype=theano.config.floatX))

test_set_y = T.cast(shared_y1, 'float64')

## tf Graph input

In [79]:
x = tf.placeholder(tf.float32, [None, n_input])

y = tf.placeholder(tf.float32, [None, n_classes])

keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
print(keep_prob)

Tensor("Placeholder_8:0", dtype=float32)


## Create some wrappers for simplicity

In [80]:
def conv2d(x, W, b, strides=1):

    # Conv2D wrapper, with bias and relu activation

    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')

    x = tf.nn.bias_add(x, b)

    return tf.nn.relu(x)

In [81]:
def maxpool2d(x, k=2):

    # MaxPool2D wrapper

    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],

                          padding='SAME')


## Create model

In [82]:
def conv_net(x, weights, biases, dropout):

    # Reshape input picture

    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    

    # Convolution Layer

    conv1 = conv2d(x, weights['wc1'], biases['bc1'])

    # Max Pooling (down-sampling)

    conv1 = maxpool2d(conv1, k=2)



    # Convolution Layer

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])

    # Max Pooling (down-sampling)

    conv2 = maxpool2d(conv2, k=2)



    # Fully connected layer

    # Reshape conv2 output to fit fully connected layer input

    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])

    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])

    fc1 = tf.nn.relu(fc1)

    # Apply Dropout

    fc1 = tf.nn.dropout(fc1, dropout)



    # Output, class prediction

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])

    return out

## Store layers weight & bias

In [83]:
weights = {

    # 5x5 conv, 1 input, 32 outputs

    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),

    # 5x5 conv, 32 inputs, 64 outputs

    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),

    # fully connected, 7*7*64 inputs, 1024 outputs

    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),

    # 1024 inputs, 10 outputs (class prediction)

    'out': tf.Variable(tf.random_normal([1024, n_classes]))

}

In [84]:
biases = {

    'bc1': tf.Variable(tf.random_normal([32])),

    'bc2': tf.Variable(tf.random_normal([64])),

    'bd1': tf.Variable(tf.random_normal([1024])),

    'out': tf.Variable(tf.random_normal([n_classes]))

}

## Construct model

In [85]:
pred = conv_net(x, weights, biases, keep_prob)

## Define loss and optimizer

In [86]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

## Evaluate model

In [87]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
print(accuracy)

Tensor("Mean_5:0", shape=(), dtype=float32)


## Initializing the variables

In [88]:
init = tf.global_variables_initializer()

valid_num = 1

## Process

In [89]:
# Launch the graph

with tf.Session() as sess:

    sess.run(init)

    step = 1

    # Keep training until reach max iterations

    print("train_try")

    while step * batch_size < training_iters:

        # print("train_try")

        #my_way

        for minibatch_index in range(n_train_batches):

            

            index = minibatch_index

            index_l = index * batch_size

            index_h = (index + 1) * batch_size



            sess.run(optimizer, feed_dict={x: train_set_x.eval()[index_l: index_h], y: train_set_y.eval()[index_l: index_h],

                                           keep_prob: dropout})

            if valid_num == 10:

                sess.run(optimizer, feed_dict={x: valid_set_x.eval()[int(index_l/2): int(index_h/2)], y: valid_set_y.eval()[int(index_l/2): int(index_h/2)],

                                           keep_prob: dropout})

                valid_num = 1



            if step % display_step == 0:

                # Calculate batch loss and accuracy

                loss, acc = sess.run([cost, accuracy], feed_dict={x: train_set_x.eval()[index_l: index_h],

                                                                  y: train_set_y.eval()[index_l: index_h],

                                                                  keep_prob: 1.})

                print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \

                      "{:.6f}".format(loss) + ", Training Accuracy= " + \

                      "{:.5f}".format(acc))

                loss, acc = sess.run([cost, accuracy], feed_dict={x: valid_set_x.eval()[int(index_l/2): int(index_h/2)],

                                                                  y: valid_set_y.eval()[int(index_l/2): int(index_h/2)],

                                                                  keep_prob: 1.})

                print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \

                      "{:.6f}".format(loss) + ", Valid Accuracy= " + \

                      "{:.5f}".format(acc))

            step += 1

            valid_num += 1



    print("Optimization Finished!")



    # Calculate accuracy for 256 mnist test images

    print("Testing Accuracy:", \

        sess.run(accuracy, feed_dict={x: test_set_x.eval()[:256],

                                      y: test_set_y.eval()[:256],

                                      keep_prob: 1.}))

train_try
Iter 1280, Minibatch Loss= 23761.435547, Training Accuracy= 0.07031
Iter 1280, Minibatch Loss= 21258.011719, Valid Accuracy= 0.09375
Iter 2560, Minibatch Loss= 14560.416992, Training Accuracy= 0.17188
Iter 2560, Minibatch Loss= 15658.187500, Valid Accuracy= 0.10938
Iter 3840, Minibatch Loss= 11073.394531, Training Accuracy= 0.15625
Iter 3840, Minibatch Loss= 13585.617188, Valid Accuracy= 0.12500
Iter 5120, Minibatch Loss= 10343.902344, Training Accuracy= 0.25000
Iter 5120, Minibatch Loss= 12755.626953, Valid Accuracy= 0.18750
Iter 6400, Minibatch Loss= 8167.491699, Training Accuracy= 0.22656
Iter 6400, Minibatch Loss= 10336.007812, Valid Accuracy= 0.17188
Iter 7680, Minibatch Loss= 6807.632812, Training Accuracy= 0.27344
Iter 7680, Minibatch Loss= 8238.911133, Valid Accuracy= 0.20312
Iter 8960, Minibatch Loss= 6044.257812, Training Accuracy= 0.29688
Iter 8960, Minibatch Loss= 7378.135742, Valid Accuracy= 0.20312
Iter 10240, Minibatch Loss= 5412.616699, Training Accuracy= 0.32

Iter 81920, Minibatch Loss= 24.236622, Training Accuracy= 0.98438
Iter 81920, Minibatch Loss= 355.462097, Valid Accuracy= 0.76562
Iter 83200, Minibatch Loss= 33.782822, Training Accuracy= 0.98438
Iter 83200, Minibatch Loss= 309.026581, Valid Accuracy= 0.79688
Iter 84480, Minibatch Loss= 35.111893, Training Accuracy= 0.98438
Iter 84480, Minibatch Loss= 292.739319, Valid Accuracy= 0.79688
Iter 85760, Minibatch Loss= 31.301935, Training Accuracy= 0.96875
Iter 85760, Minibatch Loss= 295.591064, Valid Accuracy= 0.76562
Iter 87040, Minibatch Loss= 15.666517, Training Accuracy= 0.97656
Iter 87040, Minibatch Loss= 318.450500, Valid Accuracy= 0.76562
Iter 88320, Minibatch Loss= 33.446156, Training Accuracy= 0.97656
Iter 88320, Minibatch Loss= 211.227997, Valid Accuracy= 0.82812
Iter 89600, Minibatch Loss= 30.718134, Training Accuracy= 0.97656
Iter 89600, Minibatch Loss= 220.929459, Valid Accuracy= 0.79688
Iter 90880, Minibatch Loss= 22.606049, Training Accuracy= 0.97656
Iter 90880, Minibatch Lo

Iter 162560, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 162560, Minibatch Loss= 62.986073, Valid Accuracy= 0.95312
Iter 163840, Minibatch Loss= 0.897896, Training Accuracy= 0.99219
Iter 163840, Minibatch Loss= 30.798271, Valid Accuracy= 0.95312
Iter 165120, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 165120, Minibatch Loss= 66.809166, Valid Accuracy= 0.92188
Iter 166400, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 166400, Minibatch Loss= 118.771393, Valid Accuracy= 0.90625
Iter 167680, Minibatch Loss= 2.976720, Training Accuracy= 0.99219
Iter 167680, Minibatch Loss= 92.248032, Valid Accuracy= 0.87500
Iter 168960, Minibatch Loss= 7.689154, Training Accuracy= 0.98438
Iter 168960, Minibatch Loss= 31.878410, Valid Accuracy= 0.93750
Iter 170240, Minibatch Loss= 13.438980, Training Accuracy= 0.99219
Iter 170240, Minibatch Loss= 46.828468, Valid Accuracy= 0.93750
Iter 171520, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 171520, Minibatch

Iter 244480, Minibatch Loss= 0.346364, Training Accuracy= 0.99219
Iter 244480, Minibatch Loss= 4.908966, Valid Accuracy= 0.98438
Iter 245760, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 245760, Minibatch Loss= 22.986813, Valid Accuracy= 0.98438
Iter 247040, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 247040, Minibatch Loss= 12.641784, Valid Accuracy= 0.95312
Iter 248320, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 248320, Minibatch Loss= 9.605269, Valid Accuracy= 0.96875
Iter 249600, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 249600, Minibatch Loss= 0.000000, Valid Accuracy= 1.00000
Optimization Finished!
Testing Accuracy: 0.84765625


###########################################################################################################################

# When

'wc1': tf.Variable(tf.random_normal([3, 3, 1, 32]))

### Accuracy = 0.87109375

# When 

learning rate = 0.1 ( increased from 0.001 to 0.1 )

### Accuracy =  0.125

# When

learning_rate = 0.1

training_iters = 2500

batch_size = 128

display_step = 15

### Accuracy = 0.078125

# When

training_iters = 2500

batch_size = 128

'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32]))

'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64]))

### Accuracy = 0.14453125

## Max Accuracy

## When 
learning_rate = 0.001

training_iters = 250000

batch_size = 128

display_step = 10

### Accurcy = 0.86328

# We get the Highest Accuracy when we INCREASE the ITERATION and Decrease the LEARNING RATE
### From this we get to know that the Deep Learning needs more Data as well as higer steps to learn the maximum features of the DATA.